In [2]:
!pip install yfinance --upgrade
!pip install beautifulsoup4
!pip install requests
!pip install pandas --upgrade
!pip install numpy --upgrade
!pip install plotly --upgrade
!pip install selenium
!pip install webdriver-manager

In [7]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Question 1: Extract Tesla Stock Data Using yfinance

Retrieve historical stock data for Tesla using the yfinance library and display the first five rows for review.

- yf.Ticker("TSLA") creates an object for Tesla stock history
- (period="max") downloads all historical stock data
- reset_index()'makes the date a column instead of the index
- head() shows the first 5 rows for verification

In [6]:
tesla = yf.Ticker("TSLA")
tesla_stock = tesla.history(period = "max")
tesla_stock.reset_index(inplace = True)
tesla_stock.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


# Question 2: Extract Tesla Revenue Data Using Web Scraping

Use Selenium to open the Macrotrends revenue page for Tesla, wait for the dynamic content to load, then parse the HTML with BeautifulSoup and extract the revenue table, displaying the last five rows with tail().

In [9]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [ ]:
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
driver.get(url)

In [ ]:
rows = driver.find_elements(By.XPATH, '//table[contains(@class,"historical_data_table")]/tbody/tr')

In [ ]:
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

In [ ]:
for row in rows:
    col = row.find_elements(By.TAG_NAME, "td")
    if len(col) == 2:  # only extract rows with 2 columns
        date = col[0].text
        revenue = col[1].text
        tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame({"Date":[date], "Revenue":[revenue]})], ignore_index=True)

In [23]:
driver.quit()

In [24]:
tesla_revenue.tail()

,Date,Revenue
70,2011-12-31,$39
71,2011-09-30,$58
72,2011-06-30,$58
73,2011-03-31,$49
74,2010-12-31,$36


# Question 5: Tesla Stock and Revenue Dashboard

In [40]:
def make_graph(stock_data, revenue_data, title):
    revenue_data['Date'] = pd.to_datetime(revenue_data['Date'], errors='coerce')
    revenue_data['Revenue'] = revenue_data['Revenue'].str.replace(',', '').str.replace('$','')
    revenue_data['Revenue'] = pd.to_numeric(revenue_data['Revenue'], errors='coerce')
    
    revenue_data = revenue_data.dropna(subset=['Date','Revenue'])
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x = stock_data['Date'],
        y = stock_data['Close'],
        name = "Stock Price",
        line = dict(color='blue'),
        yaxis='y1'
    ))

    fig.add_trace(go.Scatter(
        x = revenue_data['Date'],
        y = revenue_data['Revenue'],
        name = "Revenue",
        line = dict(color='red'),
        yaxis='y2'
    ))

    fig.update_layout(
        title = title,
        xaxis=dict(title='Date'),
        yaxis=dict(title='Stock Price', side='left'),
        yaxis2=dict(title='Revenue', overlaying='y', side='right'),
        legend=dict(x=0.1, y=0.9)
    )

    fig.show()

In [42]:
make_graph(tesla_stock, tesla_revenue, "Tesla Stock Price vs Revenue")